<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/Catboost_20240825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.4 MB/s eta 0:00:00


In [4]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [5]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train5.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test5.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [7]:
df_train.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,CityTier_1,CityTier_2,CityTier_3,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,PreferredPropertyStar_3.0,PreferredPropertyStar_4.0,PreferredPropertyStar_5.0,PitchSatisfactionScore_1,PitchSatisfactionScore_2,PitchSatisfactionScore_3,PitchSatisfactionScore_4,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚,FamilySize,IncomePerAge,IncomePerFamilySize,IncomePerTrips,PersonTripsProduct,IncomePerChild,LogMonthlyIncome,PitchFollowupInteraction,TripsPerAge
0,0,50.00000,1,15.0,0,1.0,4.0,5.0,1,253,1.0,0,0.0,0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,5.060000,126.5,50.6,5.0,inf,5.537334,60.0,0.100000
1,1,56.00000,0,14.0,0,1.0,4.0,2.0,1,404,0.0,1,0.0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,7.214286,404.0,202.0,2.0,inf,6.003887,56.0,0.035714
2,2,41.13181,1,10.0,1,1.0,3.0,4.0,0,278,1.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,6.758759,139.0,69.5,4.0,inf,5.631212,30.0,0.097248
3,3,37.00000,1,18.0,1,1.0,3.0,1.0,0,326,0.0,1,0.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,8.810811,326.0,326.0,1.0,inf,5.789960,54.0,0.027027
4,4,48.00000,0,17.0,1,1.0,3.0,4.0,0,258,1.0,1,0.0,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.375000,258.0,64.5,4.0,inf,5.556828,51.0,0.083333


In [8]:
# Config cell
target = "ProdTaken"

In [21]:

train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [61]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['IncomePerAge','FamilySize','IncomePerFamilySize','IncomePerTrips','PersonTripsProduct','IncomePerChild','LogMonthlyIncome','PitchFollowupInteraction','TripsPerAge']
numerical_features = default_numerical_features + added_numerical_features



#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=['id',target]
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar', 'IncomePerAge', 'FamilySize', 'IncomePerFamilySize', 'IncomePerTrips', 'PersonTripsProduct', 'IncomePerChild', 'LogMonthlyIncome', 'PitchFollowupInteraction', 'TripsPerAge']
categorical_features ['TypeofContact', 'Gender', 'Passport', 'ProdTaken', 'HasChildren', 'CityTier_1', 'CityTier_2', 'CityTier_3', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'PreferredPropertyStar_3.0', 'PreferredPropertyStar_4.0', 'PreferredPropertyStar_5.0', 'PitchSatisfactionScore_1', 'PitchSatisfactionScore_2', 'PitchSatisfactionScore_3', 'PitchSatisfactionScore_4', 'PitchSatisfactionScore_5', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior Manager', 'Designation_VP', 'MaritalStatu

In [46]:
cols_category=categorical_features

In [47]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    'depth':1,
    'l2_leaf_reg' : 8,
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [48]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [62]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x[features].iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x[features].iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

        # キャリブレーション曲線を計算
    fraction_of_positives, mean_predicted_value = calibration_curve(val_y, preds_y_proba, n_bins=10)
    fraction_of_positives_list.append(fraction_of_positives)
    mean_predicted_value_list.append(mean_predicted_value)

  # AUC
print(np.mean(list_metrics_auc), np.mean(list_metrics_f1), np.median(list_cutoff))

0:	test: 0.7353890	best: 0.7353890 (0)	total: 1.26ms	remaining: 1.26s
100:	test: 0.8530999	best: 0.8536922 (84)	total: 116ms	remaining: 1.03s
200:	test: 0.8574766	best: 0.8580361 (194)	total: 252ms	remaining: 1s
300:	test: 0.8571476	best: 0.8588259 (249)	total: 366ms	remaining: 849ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8588258523
bestIteration = 249

Shrink model to first 250 iterations.
Fold: 1, AUC: 0.8588258523101224, f1 score: 0.7250269165882482 Threshold: 0.24984737944773303
0:	test: 0.6837567	best: 0.6837567 (0)	total: 1.47ms	remaining: 1.47s
100:	test: 0.8289785	best: 0.8295051 (99)	total: 416ms	remaining: 3.7s
200:	test: 0.8373865	best: 0.8377320 (199)	total: 574ms	remaining: 2.28s
300:	test: 0.8399368	best: 0.8399368 (298)	total: 690ms	remaining: 1.6s
400:	test: 0.8402988	best: 0.8406279 (398)	total: 806ms	remaining: 1.2s
500:	test: 0.8432276	best: 0.8432276 (496)	total: 922ms	remaining: 919ms
Stopped by overfitting detector  (100 iterations wa

In [ ]:
0.8480152732367415

In [63]:
import pickle

In [66]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v5{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [67]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x799984ef07f0>, Proba: [0.01460413 0.0308366  0.04083839 ... 0.082596   0.03498036 0.00106855]
Model: <catboost.core.CatBoostClassifier object at 0x799975f2a3b0>, Proba: [0.02705541 0.06604497 0.10085142 ... 0.16531828 0.06756136 0.00183879]
Model: <catboost.core.CatBoostClassifier object at 0x799975f658a0>, Proba: [0.03739332 0.09357482 0.14971115 ... 0.24409564 0.11189741 0.0054889 ]
Model: <catboost.core.CatBoostClassifier object at 0x799975f2bd00>, Proba: [0.05000183 0.12888131 0.19330534 ... 0.33640347 0.14311479 0.00693741]
Model: <catboost.core.CatBoostClassifier object at 0x7999771ebc70>, Proba: [0.05935775 0.15529993 0.241393   ... 0.42130158 0.17902668 0.00933297]
Model: <catboost.core.CatBoostClassifier object at 0x799975f2b700>, Proba: [0.07144969 0.18204498 0.28541916 ... 0.50573511 0.20891712 0.01118566]
Model: <catboost.core.CatBoostClassifier object at 0x799985086560>, Proba: [0.08124036 0.20818682 0.32724625 ... 0.598

In [38]:
count = 10

In [68]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)